In [ ]:
import torch
import torch.nn as nn
from torchvision.models import efficientnet_b7, EfficientNet_B7_Weights
import onnxruntime as ort
import torch.nn.functional as F

class OnnxCombineModel(nn.Module):
    def __init__(self,  model_path1 , model_path2 ):
        super().__init__()

        self.session1 = ort.InferenceSession(model_path1)
        self.session2 = ort.InferenceSession(model_path2)
        self.input_names1 = [inp.name for inp in self.session1.get_inputs()]
        self.input_names2 = [inp.name for inp in self.session2.get_inputs()]

    def forward(self, image, demographics):
        print(self.input_names1)
        print(self.input_names2)
        inputs1 = {self.input_names1[0]: image, self.input_names1[1]: demographics}
        # inputs2 = {self.input_names2[0]: image} ### 43  84 model ###
        inputs2 = {self.input_names2[0]: image, self.input_names2[1]: demographics} ### 108 grose model ###
        
        outputs1 = self.session1.run(None, inputs1)
        outputs2 = self.session2.run(None, inputs2) 

        probs1 = outputs1[0].flatten()
                
        print(probs1)
        
        probs1 = F.softmax(torch.tensor(probs1) , dim = 0)
        
        probs2 = outputs2[0].flatten()  ### 43 84 model ###
        print(probs2)
        
        probs2 = F.softmax(torch.tensor(probs2) , dim = 0)
        probs = (probs1*0.5 + 0.5*probs2)

        print(probs1)
        print(probs2)
        print(probs)
        return probs

In [ ]:
MODEL_PATH_1 = "/home/mateo/cancer-ai/manager/models/2025-11-27/grose/skin_bmodel01.onnx"
MODEL_PATH_2 = "/home/mateo/cancer-ai/manager/models/2025-11-27/grose/skin_bmodel13.onnx"
MODEL_PATH_3 = "/home/mateo/cancer-ai/manager/models/2025-11-27/grose/skin_new.onnx"
IMAGE_DATA_PATH = "/home/mateo/cancer-ai/manager/dataset/dataset00014/1f2a5ba2-3e46-43e1-8b74-0068188a24bd.jpg"

In [ ]:
import numpy as np
from PIL import Image

model = OnnxCombineModel(MODEL_PATH_1, MODEL_PATH_2)
device = "cpu"
image = Image.open(IMAGE_DATA_PATH).convert("RGB")
image = image.resize((512, 512))
image = np.array(image, dtype=np.float32)
image = image * (1.0 / 255.0)

image = np.transpose(image, (2, 0, 1))
image = torch.from_numpy(image).to(device)
image = image.unsqueeze(0)
# print(image.shape)
data = torch.tensor([30, 0, 3], dtype=torch.float32).unsqueeze(0).to(device)
image = image.numpy()
data = data.numpy()
result = model(image, data)

In [ ]:
import torchvision.transforms as transforms
import onnxruntime as ort

CLASS_NAMES = [
    "AKIEC",
    "BCC", 
    "BEN_OTH",
    "BKL",
    "DF",
    "INF",
    "MAL_OTH",
    "MEL",
    "NV",
    "SCCKA",
    "VASC"
]

class ONNXInference:
    def __init__(self, model_path):
        """Initialize ONNX model session."""

        self.session = ort.InferenceSession(model_path)
        self.input_names = [inp.name for inp in self.session.get_inputs()]
        
        # Image preprocessing
        self.transform = transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def preprocess_image(self, image_path):
        """Load and preprocess image to [0,1] range as specified."""
        img = Image.open(image_path).convert('RGB')
        # Resize to 512x512
        img = img.resize((512, 512))
        # Convert to numpy array with [0,1] range
        img_array = np.array(img, dtype=np.float32)
        # Scale from [0,255] to [0,1]
        img_array = img_array * (1 / 255.0)
        # Convert to BCHW format
        img_array = np.transpose(img_array, (2, 0, 1))
        img_array = np.expand_dims(img_array, axis=0)
        return img_array
    
    def predict(self, image_path, age, gender, location):
        """Run inference on a single image with demographic data."""
        # Preprocess image
        image_tensor = self.preprocess_image(image_path)
        
        # Convert demographics to proper format
        # Gender: 'm' -> 1.0, 'f' -> 0.0
        gender_encoded = 1.0 if gender.lower() == 'm' else 0.0
        
        # Prepare demographic data as [age, gender_encoded, location]
        demo_tensor = np.array([[float(age), gender_encoded, float(location)]], dtype=np.float32)
        
        # Run inference
        inputs = {self.input_names[0]: image_tensor, self.input_names[1]: demo_tensor}
        outputs = self.session.run(None, inputs)
        
        # Model already outputs probabilities (softmax applied in forward pass)
        probs = outputs[0].flatten()
        
        # Get top 3 predictions
        top3_idx = np.argsort(probs)[-3:][::-1]
        top3 = [(CLASS_NAMES[i], float(probs[i])) for i in top3_idx]
        
        return top3

print("----------------")
onnx_model = ONNXInference("model/tricorder-3/30_new.onnx")
predictions = onnx_model.predict(f"example_dataset/dataset00092/a954cebc-6d49-4750-b485-851a307ab3fb.jpg", 30 , "f" , 3)
print(predictions)

### Combine with 2 models

In [ ]:
import onnx
import onnx_graphsurgeon as gs
import numpy as np
from onnx import shape_inference
from typing import List


def fix_reduction_nodes(graph: gs.Graph, graph_name: str = "unknown"):
    """
    Fixes ReduceL2 and ReduceMean nodes that incorrectly have axes as input (2 inputs) by moving axes to attribute.
    Searches for the Constant node producing the axes Variable and extracts its value.
    Removes the axes input and the unused Constant node after fix.
    Adds debug prints for all ReduceL2 and ReduceMean nodes.
    """
    fixed_count = 0
    removed_constants = 0
    debug_nodes = []
    for node in graph.nodes:
        if node.op in ["ReduceL2", "ReduceMean"]:
            debug_nodes.append(
                {
                    "name": node.name,
                    "op": node.op,
                    "inputs_count": len(node.inputs),
                    "inputs_types": [type(inp).__name__ for inp in node.inputs],
                    "second_input_name": (
                        node.inputs[1].name if len(node.inputs) > 1 else None
                    ),
                }
            )
            if len(node.inputs) == 2:
                data_input = node.inputs[0]
                axes_var = node.inputs[1]
                # Search for Constant node producing axes_var
                constant_node = None
                axes_values = None
                for c_node in graph.nodes:
                    if (
                        c_node.op == "Constant"
                        and c_node.outputs
                        and len(c_node.outputs) == 1
                        and c_node.outputs[0].name == axes_var.name
                    ):
                        constant_node = c_node
                        if "value" in c_node.attrs:
                            axes_values = c_node.attrs["value"].values
                            if isinstance(axes_values, np.ndarray):
                                axes_values = axes_values.tolist()
                        break
                if constant_node and axes_values is not None:
                    # Update node: remove second input, add axes attr
                    node.inputs = [data_input]
                    node.attrs["axes"] = axes_values
                    # Ensure keepdims is set (default 1 for most reductions)
                    if "keepdims" not in node.attrs:
                        node.attrs["keepdims"] = 1
                    fixed_count += 1
                    print(
                        f"[{graph_name}] Fixed {node.op} node '{node.name}': axes {axes_values} extracted from Constant '{constant_node.name}'"
                    )
                    # Mark for removal; cleanup will handle unused nodes
                    removed_constants += 1
                else:
                    print(
                        f"[{graph_name}] Warning: Could not find/extract axes for {node.op} '{node.name}'; second input '{axes_var.name}', Constant found: {constant_node is not None}"
                    )
    if debug_nodes:
        print(
            f"[{graph_name}] Total {', '.join(set(dn['op'] for dn in debug_nodes))} nodes: {len(debug_nodes)}, Fixed: {fixed_count}"
        )
        for dn in debug_nodes[:3]:  # Print first 3 for brevity
            print(
                f"  - {dn['name']}: {dn['op']}, {dn['inputs_count']} inputs, types: {dn['inputs_types']}, second_name: {dn['second_input_name']}"
            )
        if len(debug_nodes) > 3:
            print(f"  ... and {len(debug_nodes)-3} more")
    return fixed_count


def _rename_graph_tensors_and_nodes(
    graph: gs.Graph, prefix: str, skip_vars: List[gs.Variable] = None
):
    """Prefix all tensor and node names in `graph` with `prefix`, except variables in skip_vars.

    This avoids name collisions when combining multiple graphs. We compare skip_vars by object id to
    ensure we don't rename the shared input Variable object.
    """
    if skip_vars is None:
        skip_vars = []
    skip_ids = {id(v) for v in skip_vars}

    # Rename variables (tensors)
    tensors = list(graph.tensors().values())
    for var in tensors:
        if id(var) in skip_ids:
            continue
        if var.name:
            var.name = prefix + var.name

    # Rename nodes
    for node in graph.nodes:
        if node.name:
            node.name = prefix + node.name


def create_combined_onnx(model_path1, model_path2, output_path="combined.onnx"):
    """
    Combines two ONNX models into one:
    - Model1: takes 'image' and 'demographics' -> logits1
    - Model2: takes 'image' -> logits2
    - Combined: takes 'image' and 'demographics' -> (softmax(logits1) + logits2) / 2

    Key changes vs. earlier: we rename the second graph's tensors/nodes with a prefix to avoid name collisions
    and ensure the shared `image` input variable object is used by both graphs. This prevents duplicate tensor
    names and topological ordering issues during checker validation.
    """
    # Load the models
    onnx_model1 = onnx.load(model_path1)
    onnx_model2 = onnx.load(model_path2)

    # Import into graph surgeon
    graph1 = gs.import_onnx(onnx_model1)
    graph2 = gs.import_onnx(onnx_model2)

    # Fix reduction nodes in BOTH graphs for thoroughness
    total_fixed = 0
    total_fixed += fix_reduction_nodes(graph1, "Model1")
    total_fixed += fix_reduction_nodes(graph2, "Model2")
    if total_fixed == 0:
        print("No reduction fixes applied - check debug output above")

    # Rename for clarity and sharing
    image_input = graph1.inputs[0]
    image_input.name = "image"

    demographics_input = graph1.inputs[1]
    demographics_input.name = "demographics"

    # Grab model2's image input object BEFORE renaming so we can skip renaming that specific Variable
    old_image_input = graph2.inputs[0]
    old_demo_input = graph2.inputs[1]

    # Rename graph2 tensors/nodes to avoid clashes (but don't rename the image Variable object)
    _rename_graph_tensors_and_nodes(graph2, prefix='g2_', skip_vars=[old_image_input, old_demo_input])
    # _rename_graph_tensors_and_nodes(graph2, prefix="g2_", skip_vars=[old_image_input])

    # Replace all references in graph2 nodes from old_image_input to the shared image_input object
    for node in graph2.nodes:
        for i in range(len(node.inputs)):
            if node.inputs[i] is old_image_input:
                node.inputs[i] = image_input
            if node.inputs[i] is old_demo_input:
                node.inputs[i] = demographics_input

    # Update graph2's inputs list to use the shared input object (this removes a duplicate input with same name)
    graph2.inputs[0] = image_input
    graph2.inputs[1] = demographics_input

    # Get outputs (assume single output each)
    logits1 = graph1.outputs[0]
    logits1.name = "logits1"

    logits2 = graph2.outputs[0]
    logits2.name = "logits2"

    # Extract num_classes from logits1 shape (assume [batch, num_classes]; batch dynamic)
    orig_shape = logits1.shape
    if orig_shape and len(orig_shape) >= 2:
        num_classes = orig_shape[-1]
        if num_classes == 0 or num_classes is None:
            num_classes = 11  # Fallback assumption based on reported output size
        output_shape = [None, num_classes]  # Dynamic batch
    else:
        output_shape = [None, 11]  # Fallback
        num_classes = 11
        print(
            f"Warning: Could not infer num_classes from shape {orig_shape}; using fallback [None, 10]"
        )

    print(f"Inferred output shape: {output_shape}")

    # Define output variables WITH dtype and shape (no flattening)
    probs1 = gs.Variable("probs1", shape=output_shape, dtype=onnx.TensorProto.FLOAT)
    probs2 = gs.Variable("probs2", shape=output_shape, dtype=onnx.TensorProto.FLOAT)
    sum_avg = gs.Variable("sum_avg", shape=output_shape, dtype=onnx.TensorProto.FLOAT)
    avg_output1 = gs.Variable(
        "avg_output1", shape=output_shape, dtype=onnx.TensorProto.FLOAT
    )
    avg_output2 = gs.Variable(
        "avg_output2", shape=output_shape, dtype=onnx.TensorProto.FLOAT
    )

    # Softmax on first model (axis=1 for [batch, classes])
    softmax1 = gs.Node(
        op="Softmax", inputs=[logits1], outputs=[probs1], attrs={"axis": 1}
    )
    softmax2 = gs.Node(
        op="Softmax", inputs=[logits2], outputs=[probs2], attrs={"axis": 1}
    )

    # Average: (probs1 + logits2) / 2

    constant_07 = gs.Constant(
        name="constant_07", values=np.array(0.5, dtype=np.float32)
    )  # Scalar for broadcast
    constant_03 = gs.Constant(
        name="constant_03", values=np.array(0.5, dtype=np.float32)
    )  # Scalar for broadcast
    mul1 = gs.Node(
        op="Mul",  # Equivalent to / 2
        inputs=[probs1, constant_07],
        outputs=[avg_output1],
    )
    mul2 = gs.Node(
        op="Mul",  # Equivalent to / 2
        inputs=[probs2, constant_03],
        outputs=[avg_output2],
    )
    add = gs.Node(op="Add", inputs=[avg_output1, avg_output2], outputs=[sum_avg])

    graph1.cleanup()
    graph2.cleanup()
    # Combined graph: nodes from both + new nodes; inputs: image + demographics; output: avg_output
    # We put graph1 nodes first, then graph2 nodes (which we've namespaced) so producers appear before consumers.
    combined_graph = gs.Graph(
        nodes=graph1.nodes + graph2.nodes + [softmax1, softmax2, mul1, mul2, add],  #
        inputs=[image_input, demographics_input],
        outputs=[sum_avg],
    )

    # Set opset on the graph for LayerNormalization support (opset 17+)
    combined_graph.opset = 17

    # Cleanup and export - cleanup will remove unused nodes and should also fix ordering where possible
    combined_model = gs.export_onnx(combined_graph.cleanup())

    # Infer shapes to fill in any missing (helps checker)
    combined_model = shape_inference.infer_shapes(combined_model)

    # Optional: Check model
    onnx.checker.check_model(combined_model)

    # Save
    onnx.save(combined_model, output_path)
    print(f"Combined ONNX model saved to {output_path}")
    print(f"Output shape: {output_shape}")

    return combined_model


# Usage
# Note: adjust paths as needed
combined = create_combined_onnx(MODEL_PATH_1, MODEL_PATH_2, MODEL_PATH_3)

### Create model from only 1 models

In [ ]:
import onnx
import onnx_graphsurgeon as gs
import numpy as np
from onnx import shape_inference
from typing import List
import random
import numpy as np

def fix_reduction_nodes(graph: gs.Graph, graph_name: str = "unknown"):
    """
    Fixes ReduceL2 and ReduceMean nodes that incorrectly have axes as input (2 inputs) by moving axes to attribute.
    Searches for the Constant node producing the axes Variable and extracts its value.
    Removes the axes input and the unused Constant node after fix.
    Adds debug prints for all ReduceL2 and ReduceMean nodes.
    """
    fixed_count = 0
    removed_constants = 0
    debug_nodes = []
    for node in graph.nodes:
        if node.op in ['ReduceL2', 'ReduceMean']:
            debug_nodes.append({
                'name': node.name,
                'op': node.op,
                'inputs_count': len(node.inputs),
                'inputs_types': [type(inp).__name__ for inp in node.inputs],
                'second_input_name': node.inputs[1].name if len(node.inputs) > 1 else None
            })
            if len(node.inputs) == 2:
                data_input = node.inputs[0]
                axes_var = node.inputs[1]
                # Search for Constant node producing axes_var
                constant_node = None
                axes_values = None
                for c_node in graph.nodes:
                    if (c_node.op == 'Constant' and 
                        c_node.outputs and len(c_node.outputs) == 1 and 
                        c_node.outputs[0].name == axes_var.name):
                        constant_node = c_node
                        if 'value' in c_node.attrs:
                            axes_values = c_node.attrs['value'].values
                            if isinstance(axes_values, np.ndarray):
                                axes_values = axes_values.tolist()
                        break
                if constant_node and axes_values is not None:
                    # Update node: remove second input, add axes attr
                    node.inputs = [data_input]
                    node.attrs['axes'] = axes_values
                    # Ensure keepdims is set (default 1 for most reductions)
                    if 'keepdims' not in node.attrs:
                        node.attrs['keepdims'] = 1
                    fixed_count += 1
                    print(f"[{graph_name}] Fixed {node.op} node '{node.name}': axes {axes_values} extracted from Constant '{constant_node.name}'")
                    # Mark for removal; cleanup will handle unused nodes
                    removed_constants += 1
                else:
                    print(f"[{graph_name}] Warning: Could not find/extract axes for {node.op} '{node.name}'; second input '{axes_var.name}', Constant found: {constant_node is not None}")
    if debug_nodes:
        print(f"[{graph_name}] Total {', '.join(set(dn['op'] for dn in debug_nodes))} nodes: {len(debug_nodes)}, Fixed: {fixed_count}")
        for dn in debug_nodes[:3]:  # Print first 3 for brevity
            print(f"  - {dn['name']}: {dn['op']}, {dn['inputs_count']} inputs, types: {dn['inputs_types']}, second_name: {dn['second_input_name']}")
        if len(debug_nodes) > 3:
            print(f"  ... and {len(debug_nodes)-3} more")
    return fixed_count


# def _rename_graph_tensors_and_nodes(graph: gs.Graph, prefix: str, skip_vars: List[gs.Variable] = None):
#     """Prefix all tensor and node names in `graph` with `prefix`, except variables in skip_vars.

#     This avoids name collisions when combining multiple graphs. We compare skip_vars by object id to
#     ensure we don't rename the shared input Variable object.
#     """
#     if skip_vars is None:
#         skip_vars = []
#     skip_ids = {id(v) for v in skip_vars}

#     # Rename variables (tensors)
#     tensors = list(graph.tensors().values())
#     for var in tensors:
#         if id(var) in skip_ids:
#             continue
#         if var.name:
#             var.name = prefix + var.name

#     # Rename nodes
#     for node in graph.nodes:
#         if node.name:
#             node.name = prefix + node.name


def create_combined_onnx(model_path1, output_path='combined.onnx'):
    """
    Combines two ONNX models into one:
    - Model1: takes 'image' and 'demographics' -> logits1
    - Model2: takes 'image' -> logits2
    - Combined: takes 'image' and 'demographics' -> (softmax(logits1) + logits2) / 2

    Key changes vs. earlier: we rename the second graph's tensors/nodes with a prefix to avoid name collisions
    and ensure the shared `image` input variable object is used by both graphs. This prevents duplicate tensor
    names and topological ordering issues during checker validation.
        """
    # Load the models
    onnx_model1 = onnx.load(model_path1)
    # onnx_model2 = onnx.load(model_path2)

    # Import into graph surgeon
    graph1 = gs.import_onnx(onnx_model1)
    # graph2 = gs.import_onnx(onnx_model2)

    # Fix reduction nodes in BOTH graphs for thoroughness
    total_fixed = 0
    total_fixed += fix_reduction_nodes(graph1, "Model1")
    # total_fixed += fix_reduction_nodes(graph2, "Model2")
    if total_fixed == 0:
        print("No reduction fixes applied - check debug output above")

    # Rename for clarity and sharing
    image_input = graph1.inputs[0]
    image_input.name = 'image'

    demographics_input = graph1.inputs[1]
    demographics_input.name = 'demographics'

    # Grab model2's image input object BEFORE renaming so we can skip renaming that specific Variable
    # old_image_input = graph2.inputs[0]
    # old_demo_input = graph2.inputs[1]

    # Rename graph2 tensors/nodes to avoid clashes (but don't rename the image Variable object)
    # _rename_graph_tensors_and_nodes(graph2, prefix='g2_', skip_vars=[old_image_input, old_demo_input])
    # _rename_graph_tensors_and_nodes(graph2, prefix='g2_', skip_vars=[old_image_input])

    # Replace all references in graph2 nodes from old_image_input to the shared image_input object
    # for node in graph2.nodes:
    #     for i in range(len(node.inputs)):
    #         if node.inputs[i] is old_image_input:
    #             node.inputs[i] = image_input
            # if node.inputs[i] is old_demo_input:
            #     node.inputs[i] = demographics_input

    # Update graph2's inputs list to use the shared input object (this removes a duplicate input with same name)
    # graph2.inputs[0] = image_input
    # graph2.inputs[1] = demographics_input

    # Get outputs (assume single output each)
    logits1 = graph1.outputs[0]
    logits1.name = 'logits1'
    
    # logits2 = graph2.outputs[0]
    # logits2.name = 'logits2'

    # Extract num_classes from logits1 shape (assume [batch, num_classes]; batch dynamic)
    orig_shape = logits1.shape
    print(orig_shape)
    if orig_shape and len(orig_shape) >= 2:
        num_classes = orig_shape[-1]
        if num_classes == 0 or num_classes is None:
            num_classes = 11  # Fallback assumption based on reported output size
        output_shape = [None, num_classes]  # Dynamic batch
    else:
        output_shape = [None, 11]  # Fallback
        num_classes = 11
        print(f"Warning: Could not infer num_classes from shape {orig_shape}; using fallback [None, 10]")

    print(f"Inferred output shape: {output_shape}")

    # Define output variables WITH dtype and shape (no flattening)
    probs1 = gs.Variable('probs1', shape=output_shape, dtype=onnx.TensorProto.FLOAT)
    # probs2 = gs.Variable('probs2', shape=output_shape, dtype=onnx.TensorProto.FLOAT)
    sum_avg = gs.Variable('sum_avg', shape=output_shape, dtype=onnx.TensorProto.FLOAT)
    avg_output = gs.Variable('avg_output', shape=output_shape, dtype=onnx.TensorProto.FLOAT)
    # avg_output2 = gs.Variable('avg_output2', shape=output_shape, dtype=onnx.TensorProto.FLOAT)

    # Softmax on first model (axis=1 for [batch, classes])

    # softmax2 = gs.Node(
    #     op='Softmax',
    #     inputs=[logits2],
    #     outputs=[probs2],
    #     attrs={'axis': 1}
    # )

    # Average: (probs1 + logits2) / 2
    rand = random.uniform(1,2)
    print(rand)
    constant_rand = gs.Constant(name='constant_rand', values=np.array(rand, dtype=np.float32))  # Scalar for broadcast
    # constant_03 = gs.Constant(name='constant_03', values=np.array(0.5, dtype=np.float32))  # Scalar for broadcast
    mul = gs.Node(
        op='Mul',  # Equivalent to / 2
        inputs=[logits1, constant_rand],
        outputs=[avg_output]
    )
    softmax = gs.Node(
        op='Softmax',
        inputs=[avg_output],
        outputs=[probs1],
        attrs={'axis': 1}
    )    
    # mul2 = gs.Node(
    #     op='Mul',  # Equivalent to / 2
    #     inputs=[probs2, constant_03],
    #     outputs=[avg_output2]
    # )
    # add = gs.Node(
    #     op='Add',
    #     inputs=[avg_output1, avg_output2],
    #     outputs=[sum_avg]
    # )

    # Combined graph: nodes from both + new nodes; inputs: image + demographics; output: avg_output
    # We put graph1 nodes first, then graph2 nodes (which we've namespaced) so producers appear before consumers.
    combined_graph = gs.Graph(
        nodes=graph1.nodes + [mul, softmax],
        inputs=[image_input, demographics_input],
        outputs=[probs1]
    )

    # Set opset on the graph for LayerNormalization support (opset 17+)
    combined_graph.opset = 17

    # Cleanup and export - cleanup will remove unused nodes and should also fix ordering where possible
    combined_model = gs.export_onnx(combined_graph.cleanup())

    # Infer shapes to fill in any missing (helps checker)
    combined_model = shape_inference.infer_shapes(combined_model)

    # Optional: Check model
    onnx.checker.check_model(combined_model)

    # Save
    onnx.save(combined_model, output_path)
    print(f"Combined ONNX model saved to {output_path}")
    print(f"Output shape: {output_shape}")

    return combined_model

# Usage
# Note: adjust paths as needed
combined = create_combined_onnx(MODEL_PATH_1, MODEL_PATH_3)


In [ ]:
import onnx

# Load the original model
model = onnx.load("model/tricorder-3/30_new.onnx")

# Check original details (optional: for debugging)
print("Original IR version:", model.ir_version)
print("Original opset versions:", [(imp.domain, imp.version) for imp in model.opset_import])

# Downgrade IR version to 11 (your runtime's max)
model.ir_version = 10

# Save the downgraded model
downgraded_path = "model/tricorder-3/30_new.onnx"
onnx.save(model, downgraded_path)
print(f"Downgraded model saved to: {downgraded_path}")